#  Import Libraries

In [7]:
# Import the essential packages 
import requests
import json
import pandas as pd
import os
import numpy as np
import random
import cv2

#  a.)  Text input to use to search Amazon 

# b.)  The key data returned by the API into a dataframe

In [ ]:

params = {
    'api_key': '0F5DAC23BEB543C1B26A6CD61B44A5E9',
    'type': 'category',
    'url': 'https://www.amazon.com/s?bbn=16225009011&rh=n%3A%2116225009011%2Cn%3A502394%2Cn%3A281052',
    'page': '1'
    }
  # make the http GET request to Rainforest API
api_result = requests.get('https://api.rainforestapi.com/request', params)
api_response = api_result.json()
data1 = pd.DataFrame(api_response['category_results'])

  
for x in range(1,11):
    params = {
    'api_key': 'F1C8601D5CC5445EB858CC86A4F6DBF8',
    'type': 'category',
    'url': 'https://www.amazon.com/s?bbn=16225009011&rh=n%3A%2116225009011%2Cn%3A502394%2Cn%3A281052',
    'page': x
    }
   # make the http GET request to Rainforest API
  #print(x)
    api_result = requests.get('https://api.rainforestapi.com/request', params)
    api_response = api_result.json()
  
  # Read the file in to dataframe
  
    data = pd.DataFrame(api_response['category_results'])
    data1 = data1.append(data,ignore_index=True)

  


In the above chunck of code, Rainforest API has been called to search amazon product called digital cameras. In the above code, 10 pages was fetched on Amazon using rainforest API

In [ ]:
# Check the shape of the dataframe
data1.shape

In [ ]:
# Display the first few rows of the dataframe
data1.head()

In [ ]:
# the basic information about the variable in the dataframe
data1.info()

There are 312 obsevations and 18 columns in the dataframe. Product name, Product image url and Price are the important variables for futher task for this project.

# Remove non-Amazon prime products 

In [ ]:
# Filter the data which is not Amazon prime
data2 = data1.loc[data1['is_prime'] == True]
data2.shape

In [ ]:
# Pre- process the price column to get price 
df3 = data2['price'].apply(pd.Series)
df3.head()
df3.shape

In [ ]:
# Merge the two dataframe 
merged_data = pd.merge(data2, df3, left_index=True, right_index=True)
merged_data.head()


In [ ]:
# Reanme the value column into price_item

merged_data.rename({'value': 'price_item'}, axis=1, inplace=True)
merged_data.head()

# Sort the rest by price

In [ ]:
# Sort the dataframe by price_item

merged_data1 = merged_data.sort_values(by='price_item')
merged_data1.head()

# Save the dataframe as a csv

In [ ]:
merged_data1.to_csv('Amazon_data.csv')

# Get the product image for each product in the dataframe using the image url and save as a jpg to a local folder

In [ ]:
product = pd.read_csv('A.csv')
urls1 = product['image']
urls1.head()

In [ ]:
# change the directory
os.chdir('C:\\Users\\01-18-20\\Documents\\Job_Applications\\CV_New\\NHS\\image_folder')


In [ ]:
images = []
for img in urls1:
    # We can split the file based upon / and extract the last split within the python list below:
    file_name = img.split('/')[-1]
    
    print(f"This is the file name: {file_name}")
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        images.append(img)


All of the images are stored in the local folder called image_folder

#  Challenge

# # Combine several products into one combined image montage 

In [ ]:
os.chdir("C:\\Users\\01-18-20\\Documents\\Job_Applications\\CV_New\\NHS\\image_folder")
image_paths = [os.path.abspath(i) for i in os.listdir()]
print(image_paths)

#print(f"{len(image_paths)} image(s) found!")
rows = int(input("Enter the number of rows: "))
columns = int(input("Enter the number of columns: "))        
size = 300
for image_path in image_paths:
    image = cv2.imread(image_path)
    image_size = image.shape[0]
    if image_size <size:
        size = image_size
counter = 0
image_rows = []
for row in range(rows):
    for column in range(columns):
        image = cv2.imread(image_paths[counter])
        image = cv2.resize(image,(size, size))
        if column == 0:
            image_list = image
        else:
            image_list = np.concatenate((image_list, image), axis = 1)
        counter += 1     
    image_rows.append(image_list)
montage = np.concatenate(image_rows, axis=0)
      
        
    

In [19]:
# Save the montage image
# cv2.imwrite("montage_test2.jpg", montage)

## Some images metrics such as brightness/contrast.

In [ ]:
# Load  the image
image_bright = cv2.imread("C:\\Users\\01-18-20\\Documents\\Job_Applications\\CV_New\\NHS\\image_folder\\51JSQ8NaX3L._AC_UL320_.jpg")
cv2.imshow("Original",image_bright)

# Matrix of ones which is multiplied by a scaler value of 60
intensity_matrix = np.ones(image_bright.shape, dtype = "uint8")*60

# Add intensity matrix to input image in order to increase the brightness
brighted_image = cv2.add(image_bright, intensity_matrix)
cv2.imshow("Bright", brighted_image)

# Subtract intensity matrix from input image in order to decrease the brightness
drakened_image = cv2.subtract(image_bright, intensity_matrix)
cv2.imshow("Dark", drakened_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Save the images
cv2.imwrite("bright_test.jpg", brighted_image)
cv2.imwrite("drakened_test.jpg", drakened_image)
